In [1]:
from data.process import *

2024-07-25 18:00:23.162946: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 18:00:23.170362: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 18:00:23.180987: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 18:00:23.181022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 18:00:23.190072: I tensorflow/core/platform/cpu_feature_gua

In [2]:
train_df, dev_df, test_df = retrieve_train_dev_test_as_category_for_xgboost()
X_train, Y_train = split_input_output(train_df)
X_dev, Y_dev = split_input_output(dev_df)

In [3]:
from lightgbm import LGBMClassifier, log_evaluation
from model.lgbm_opt import *

/home/gakim/gabyproj/InsuranceCrossSelling/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
lgbm_params = {
    **lgbm_fixed_params,
    # categorical features
    'enable_categorical': True,
    # fixed after first optimization
    'colsample_bytree': 0.25,
    'learning_rate': 0.01,

    # fixed after third optimization
    'max_depth': 128,
    'num_leaves': 4096,
    'min_data_in_leaf': 25,

    # override fixed params
    'n_estimators': 10000,
    'early_stopping_rounds': 200,
}

In [5]:
cat_features = list(X_train.select_dtypes(include=['category']).columns)

In [6]:
model = LGBMClassifier(
    **lgbm_params,
)
log_callback = log_evaluation(period=100)
boosting = model.fit(X_train, Y_train, eval_set=[(X_dev, Y_dev)], categorical_feature=cat_features, eval_metric='auc', callbacks=[log_callback])

[100]	valid_0's auc: 0.876504	valid_0's binary_logloss: 0.30036
[200]	valid_0's auc: 0.88045	valid_0's binary_logloss: 0.274283
[300]	valid_0's auc: 0.884047	valid_0's binary_logloss: 0.2608
[400]	valid_0's auc: 0.886025	valid_0's binary_logloss: 0.253787
[500]	valid_0's auc: 0.887225	valid_0's binary_logloss: 0.250053
[600]	valid_0's auc: 0.888296	valid_0's binary_logloss: 0.247504
[700]	valid_0's auc: 0.8892	valid_0's binary_logloss: 0.245768
[800]	valid_0's auc: 0.889882	valid_0's binary_logloss: 0.244558
[900]	valid_0's auc: 0.89035	valid_0's binary_logloss: 0.243716
[1000]	valid_0's auc: 0.890697	valid_0's binary_logloss: 0.243127
[1100]	valid_0's auc: 0.890963	valid_0's binary_logloss: 0.242719
[1200]	valid_0's auc: 0.891228	valid_0's binary_logloss: 0.242372
[1300]	valid_0's auc: 0.891404	valid_0's binary_logloss: 0.242127
[1400]	valid_0's auc: 0.891533	valid_0's binary_logloss: 0.241941
[1500]	valid_0's auc: 0.891689	valid_0's binary_logloss: 0.241757
[1600]	valid_0's auc: 0.89

In [7]:
import pprint
pprint.pprint(model.best_score_)

defaultdict(<class 'collections.OrderedDict'>,
            {'valid_0': OrderedDict([('auc', 0.892542157392671),
                                     ('binary_logloss', 0.24079890991068958)])})


In [8]:
lgbm_output = model.predict_proba(test_df, iteration_range=(0, model._best_iteration + 1))[:, 1]